In [21]:
# Import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

# create the Spark Session
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()

# # create the Spark Context
# sc = spark.sparkContext

In [10]:
friend = sc.textFile('hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt')

In [14]:
friend.collect()[:3]

['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878']

In [37]:
friend = spark.read.csv("hw1-bundle/hw1-bundle/q1/data/soc-LiveJournal1Adj.txt", sep='\t')
friend = friend.select(col("_c0").alias("User"), col("_c1").alias("Friends"))
friend.show(5)

+----+--------------------+
|User|             Friends|
+----+--------------------+
|   0|1,2,3,4,5,6,7,8,9...|
|   1|0,5,20,135,2409,8...|
|   2|0,117,135,1220,27...|
|   3|0,12,41,55,1532,1...|
|   4|0,8,14,15,18,27,7...|
+----+--------------------+
only showing top 5 rows



In [24]:
split_by_comma = lambda x: x.split(',')
split_by_comma = udf(split_by_comma, StringType())

In [35]:
all_user_id = set(range(friend.count()))

In [38]:
unfriend = friend.withColumn('Friends', all_user_id - set(split('Friends', ',')))

PySparkTypeError: [NOT_ITERABLE] Column is not iterable.

In [30]:
friend.show(5)

+----+--------------------+
|User|             Friends|
+----+--------------------+
|   0|[1, 2, 3, 4, 5, 6...|
|   1|[0, 5, 20, 135, 2...|
|   2|[0, 117, 135, 122...|
|   3|[0, 12, 41, 55, 1...|
|   4|[0, 8, 14, 15, 18...|
+----+--------------------+
only showing top 5 rows



In [8]:
friend.select("Friends").take(10)

[Row(Friends='1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94'),
 Row(Friends='0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592'),
 Row(Friends='0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878'),
 Row(Friends='0,12,41,55,1532,12636,13185,27552,38737'),
 Row(Friends='0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822'),
 Row(Friends='0,1,20,2022,22939,23527,30257,32503,35633

In [ ]:
unfriend = 

In [2]:
findspark.find()

'c:\\Users\\Wenhao\\miniconda3\\envs\\cse-547\\Lib\\site-packages\\pyspark'